# Korean Hate Speech Detection

From Kaggle competition: https://www.kaggle.com/c/korean-hate-speech-detection

## 0. Import Required Libraries

In [144]:
import numpy as np
import pandas as pd



## 1. Check Data

In [145]:
df_train = pd.read_csv('./dataset/train.hate.csv')
df_validation = pd.read_csv('./dataset/dev.hate.csv')
df_test = pd.read_csv('./dataset/test.hate.no_label.csv')

dataframes = [df_train, df_validation, df_test]

for df in dataframes:
    print(df.head)



<bound method NDFrame.head of                                                comments label
0     (현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...  hate
1     ....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...  none
2     ...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...  hate
3                    1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데  none
4     1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...  hate
...                                                 ...   ...
7891                                      힘내세요~ 응원합니다!!  none
7892                             힘내세요~~삼가 고인의 명복을 빕니다..  none
7893                              힘내세용 ^^ 항상 응원합니닷 ^^ !  none
7894  힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...  none
7895                                 힘들면 관뒀어야지 그게 현명한거다  none

[7896 rows x 2 columns]>
<bound method NDFrame.head of                                               comments      label
0                          송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.       none
1                                     

## 2. Data Preprocessing

### 2.1 Remove unwanted part from comment

1. 각종 이모티콘, 특수문자 등을 제거하여 효율성 증대
2. 반복되는 문자를 동일하게 처리해서 효율성 증대 ex. ㅋㅋㅋㅋㅋㅋㅋㅋ -> ㅋㅋ

In [146]:
import re

regular_expression1 = "[^a-zA-Z0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]"
regular_expression2 = "ㅋ{3,}"

for df in dataframes:
    df['comments'] = df['comments'].str.replace(regular_expression1, "")
    df['comments'] = df['comments'].str.replace(regular_expression2, "ㅋㅋ")
    
    #print(df.head)
        


<ipython-input-146-190417025c3c>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comments'] = df['comments'].str.replace(regular_expression1, "")
<ipython-input-146-190417025c3c>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comments'] = df['comments'].str.replace(regular_expression2, "ㅋㅋ")


### 2.2 Spell check

인터넷 댓글이기 때문에, 맞춤법이 정확하지 않아 토큰화가 제대로 되지 않는 경우가 있다고 판단했음.

이에 따라, 맞춤법 교정을 진행

단점 및 부작용: 제대로 교정되지 않을 수 있음, 인터넷 유행어나 신조어 등이 제대로 분석되지 않을 수 있음

In [147]:
from hanspell import spell_checker

for df in dataframes:
    for i in range(0, df.shape[0]):
        result = spell_checker.check(df['comments'][i])
        #print('Comment_' + str(i+1) + ':          ' + df['comments'][i])
        #print('Comment_' + str(i+1) + '_Refined:  ' + result.checked)
        df['comments'][i] = result.checked
    print(df)

                                               comments label
0     현재 호텔 주인 심정 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 누군...  hate
1     한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤의 슬픔을 미처 ...  none
2     못된 놈들 남의 고통을 즐겼던 놈들 이젠 마땅한 처벌을 받아야지 그래야 공정한 사회...  hate
3                      12화 어설펐는데 34화 지나서부터는 갈수록 너무 재밌던데  none
4     1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 에 걸리 안 들...  hate
...                                                 ...   ...
7891                                         힘내세요 응원합니다  none
7892                                힘내세요 삼가 고인의 명복을 빕니다  none
7893                                   힘내세요  항상 응원합니다    none
7894  힘내소 연기로 답해요 나도 53살인데 이런 일 저런 일 다 있더라고요 인격을 믿습니다홨팅  none
7895                                힘들면 관뒀어야지 그게 현명한 거다  none

[7896 rows x 2 columns]
                                              comments      label
0                          송중기 시대극은 믿고 본다 첫 회 신선하고 좋았다       none
1                                             지현우 나쁜 놈  offensive
2    알바쓰고많이만들면되지 돈 욕심 없으면 골목식당 왜 

### 2.3 Tokenize

데이터를 토큰화해 모델에 입력으로 사용하므로, 한국어 형태소 분석기를 통해 토큰화

조사 등의 불용어를 정의하고 이를 제거

In [143]:
from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Komoran
from konlpy.tag import Mecab

okt = Okt()

stopwords=['뭐','으면','을','의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

for df in dataframes:
    for i in range(0, df.shape[0]):
        #print('<Comment_' + str(i+1) + '>' + df['comments'][i])
        #print('Comment_' + str(i+1) + ':          ' + df['comments'][i])
        #print('Comment_' + str(i+1) + '_Refined:  ' + result.checked)
        tmp = okt.morphs(df['comments'][i])
        result = []
        for word in tmp:
            if word not in stopwords:
                result.append(word)

        df['comments'][i] = result
    
    print(df)

    

TypeError: No matching overloads found for kr.lucypark.okt.OktInterface.tokenize(list,java.lang.Boolean,java.lang.Boolean), options are:
	public java.util.List kr.lucypark.okt.OktInterface.tokenize(java.lang.String,java.lang.Boolean,java.lang.Boolean)



### 2.4 Integer encoding

## 9. Export result to csv format 

In [49]:
export_test = df_test
result = np.zeros((974, 1), dtype=np.int64)

for i in range(0, result.shape[0]):
    result[i] = i % 3

In [50]:
export_test['label'] = result

for i in range(0, result.shape[0]):
    export_test['comments'][i] = "comment_" + str(i+1)

export_test.to_csv('./export.csv', sep=',', index=False)
    

<ipython-input-50-781d37f451a8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  export_test['comments'][i] = "comment_" + str(i+1)
